<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/train_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [24]:
import tensorflow
from keras.models import Model

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Model

In [14]:
from utils import *
from dataset import train_val_split,FashionDataset

In [7]:
#Loading the labels and removing missing images:
data=pd.read_csv('/content/styles.csv',usecols=["id","gender","masterCategory",
                                                 "subCategory","articleType",
                                                 "baseColour","season","usage"])

In [8]:
# training and validation dataset
dataset = FashionDataset(data,labels=["gender","masterCategory","subCategory"])

[INFO]: Checking if all images are present


44446it [00:03, 13479.12it/s]


[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [9]:
dataset.df.head()

,id,classes,index
0,15970.jpg,"(Men, Apparel, Topwear)",0
1,39386.jpg,"(Men, Apparel, Bottomwear)",1
2,59263.jpg,"(Women, Accessories, Watches)",2
3,21379.jpg,"(Men, Apparel, Bottomwear)",3
4,53759.jpg,"(Men, Apparel, Topwear)",4


In [10]:
train_data, val_data = train_val_split(dataset.df)
print(f"[INFO]: Number of training sampels: {len(train_data)}")
print(f"[INFO]: Number of validation sampels: {len(val_data)}")

[INFO]: Number of training sampels: 39996
[INFO]: Number of validation sampels: 4444


In [11]:
batch_size = 32
# training and validation data loader
train_generator = dataset.datagen.flow_from_dataframe(
    train_data, batch_size=batch_size,directory='./images/',
    x_col='id',y_col='index',class_mode='other',color_mode="rgb",
    target_size=(224, 224),shuffle=True
)
val_generator = dataset.datagen.flow_from_dataframe(
    val_data, batch_size=batch_size,directory='./images/',
    x_col='id',y_col='index',class_mode='other',color_mode="rgb",
    target_size=(224, 224),shuffle=True
)

Found 39996 validated image filenames.
Found 4444 validated image filenames.


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
# Fit a MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(dataset.df['classes'].values.tolist())

MultiLabelBinarizer()

In [20]:
len(mlb.classes_)

56

In [16]:
train_multilabel_generator = multilabel_flow_from_dataframe(train_generator, mlb, train_data)
val_multilabel_generator = multilabel_flow_from_dataframe(val_generator, mlb, val_data)

In [18]:
# Load model architecture
model_file = open("./myModel_arc.json","r")
loaded_json_model = model_file.read()
model_file.close()
loaded_model = model_from_json(loaded_json_model)

In [31]:
model = Model(inputs=loaded_model.input,outputs=loaded_model.layers[len(loaded_model.layers)-2].output)

In [33]:
num_classes = len(mlb.classes_)

In [34]:
final_model = tensorflow.keras.Sequential([
    model,
    tensorflow.keras.layers.Dense(num_classes,activation="softmax",name="fc"+str(num_classes))
])

In [36]:
lr = 0.001
optim = tensorflow.keras.optimizers.RMSprop(learning_rate=lr)
epochs = 20
loss_func = "binary_crossentropy"

In [37]:
final_model.compile(optimizer=optim,loss = loss_func,metrics=["accuracy"])

In [39]:
history = final_model.fit_generator(train_multilabel_generator,epochs=epochs,validation_data=val_multilabel_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


StopIteration: ignored